### Time varying GSTC - 1D FDTD

In [33]:
from Electromagnetics.FDTD import *
import numpy as np
import time, sys

n_dim = 1
r0, r1, dr = [None]*3
S = 0.95
n_t = None
save_every = 10
omega = 1.0

L_0 = 2.0*np.pi/omega
L_tot = 10.0*L_0

r0 = np.array([0.0])
r1 = np.array([L_tot])
dr = np.array([L_0/100])
    
dt = S*dr[0]
d_v = np.prod(dr)
n_t = 2000

## fdtd simulator
fdtd = FDTDSimulator()

fdtd.SetSimulationBox(r0, r1, dr, dt)
fdtd.save_every = save_every 

## epsilon_r
#eps_r = 1.0
#eps_args = {'r0':r0, 'r1':r1, 'mag_in':eps_r, 'mag_out':1.0}
#fdtd.AddMaterial({'type':FVTypes.EpsIsoBox, 'name':'eps_r', 'args':eps_args})


## J_e
alpha = 0.5
t_0 = 6.0/alpha
t_1 = t_0*1.4
f_t = lambda t: -np.exp(-alpha**2*(t-t_0)**2)# + np.exp(-alpha**2*(t-t_1)**2)

r_j0 = r0 + 1.0*L_0
je_mag = 1.0/dr[0]
je_args = {'r0':r_j0, 'mag':je_mag, 'f_t':f_t, 'src_dir':'z'}
fdtd.AddSources({'type':FVTypes.JePoint, 'name':'Je', 'args':je_args})

## J_m
jm_args = {'r0':r_j0, 'mag':-je_mag, 'f_t':f_t, 'src_dir':'y'}
fdtd.AddSources({'type':FVTypes.JmPoint, 'name':'Jm', 'args':jm_args})

## metasurface sources
## J_e
dL_ms = 4.0*L_0
dt_ms = dL_ms
#g_t = lambda t: -2.0*alpha**2*(t-t_0)*np.exp(-alpha**2*(t-t_0)**2) +2.0*alpha**2*(t-t_1)*np.exp(-alpha**2*(t-t_1)**2)
g_t = lambda t: 0.0#np.exp(-alpha**2*(t-t_0)**2) - np.exp(-alpha**2*(t-t_1)**2)

f_t_ms = lambda t: -f_t(t - dt_ms) + g_t(t - dt_ms)

r_j1 = r_j0 + dL_ms
je_args = {'r0':r_j1, 'mag':je_mag, 'f_t':f_t_ms, 'src_dir':'z'}
fdtd.AddSources({'type':FVTypes.JePoint, 'name':'Je', 'args':je_args})

## J_m
jm_args = {'r0':r_j1, 'mag':-je_mag, 'f_t':f_t_ms, 'src_dir':'y'}
fdtd.AddSources({'type':FVTypes.JmPoint, 'name':'Jm', 'args':jm_args})


## pml
d_pml = np.array([0.9*L_0])
s_pml = np.array([1.0+3.0j/L_0])
pml_args = {'d_pml':d_pml, 's_pml':s_pml}
fdtd.AddPML({'type':FVTypes.UPML, 'name':'pml', 'args':pml_args})


## view planes
fdtd.AddViewPlane({'type':FVTypes.VPSide, 'r':r0, 'args':{'A':'E', 'A_dir':'z', 'O_dir':None, 'name':'E'}})        

fdtd.StepFields(n_t)
n_saved = fdtd.n_saved

x_ms = r_j1[0]
 

simulation time: 0:1


In [34]:
%matplotlib wx
    
import matplotlib.pyplot as plt

r_E, E_out_list = fdtd.GetOutputs('E')
x_E = r_E[0]
e_max = [np.max(np.abs(e)) for e in E_out_list]
print('max(e_max):', max(e_max))
e_max = max(e_max)
N_x = len(x_E)

##---
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib

save_anim = False
file_format = 'gif'   ## 'gif' / 'mp4'

use_subplots = False

font = {'family' : 'serif', 'weight' : 'normal', 'size'   : 14}
matplotlib.rc('font', **font)

fig, axarr = None, None
def animate_E(i):
    plt.clf()
    fig = plt.plot(x_E, E_out_list[i], 'b')
    ax = plt.gca()
    ax.set_ylim([-e_max, e_max])
    plt.axvline(x_ms, color= 'r', linestyle='-.')
    return fig


fig = plt.figure()
anim = animation.FuncAnimation(fig, animate_E, frames=n_saved, interval=100)

if save_anim:
    if file_format=='gif':
        anim.save('other/e-1d.gif', writer="imagemagick", fps=15, dpi=200)
    elif file_format=='mp4':
        anim.save('other/e-1d.mp4', writer="mencoder", fps=15, dpi=200)

plt.show()



max(e_max): 1.00777414437
